# TAR: Proyecto Final 

In [1]:
%pip install gymnasium pandas numpy

Note: you may need to restart the kernel to use updated packages.


## Tratamiento de datos

In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
ruta_datosMOP = "./Datos/MOP"
ruta_datosADME = "./Datos/ADME"
ruta_datos_procesados = "./Datos_procesados"

## Entorno personalizado para el problema hidro-térmico

In [6]:
import gymnasium as gym
from gymnasium import spaces

class HydroThermalEnv(gym.Env):
    T_MAX = 52
    N_HIDRO = 5

    P_BON_MAX = 155
    P_BAY_MAX = 108
    P_PAL_MAX = 333
    P_SAL_MAX = 1890
    P_CLAIRE_MAX = P_BON_MAX + P_BAY_MAX + P_PAL_MAX + P_SAL_MAX

    P_SOLAR = 254
    P_EOLICO = 1584.7
    P_BIOMASA = 487.3

    Q_BON_MAX = 680
    Q_BAY_MAX = 828
    Q_PAL_MAX = 1372
    Q_SAL_MAX = 4200
    Q_CLAIRE_MAX = Q_BON_MAX + Q_BAY_MAX + Q_PAL_MAX + Q_SAL_MAX

    V_BON_MAX = 8200
    V_BAY_MAX = 0
    V_PAL_MAX = 1300
    V_SAL_MAX = 1500
    V_CLAIRE_MAX = V_BON_MAX + V_BAY_MAX + V_PAL_MAX + V_SAL_MAX

    K_CLAIRE = P_CLAIRE_MAX / Q_CLAIRE_MAX

    V0 = V_CLAIRE_MAX / 2

    def _inicial_hidrologia(self): ...

    def _siguiente_hidrologia(self, tiempo, hidrologia_actual): ...

    def _vertido(self, qt):
        return np.max(self.v - qt + self._aportes() - self.V_CLAIRE_MAX, 0)
    
    def _aportes(self): return 0
    
    def _demanda(self): return 0
    
    def _gen_eolico(self): return 0

    def _gen_solar(self): return 0

    def _gen_bio(self): return 0

    def _gen_termico_barato(self, demanda_residual): return 0

    def _gen_termico_caro(self, demanda_residual): return 0

    def _despachar(self, qt):
        demanda_residual = self._demanda() - self._gen_eolico() - self._gen_solar() - self._gen_bio() - (self.K_CLAIRE * qt)
        energia_termico_barato = 0
        energia_termico_caro = 0
        exportacion = 0

        if demanda_residual > 0:
            # Primero uso termico barato
            energia_termico_barato = self._gen_termico_barato(demanda_residual)
            demanda_residual -= energia_termico_barato

            if demanda_residual > 0:
                energia_termico_caro = self._gen_termico_caro(demanda_residual)
                demanda_residual -= energia_termico_caro

        if demanda_residual < 0:
            exportacion = -demanda_residual

        # Ajustar valores de ingreso por exportacion y costos de térmicos
        return exportacion * 50, energia_termico_barato * 100 + energia_termico_caro * 200

    def __init__(self):
        # Defino espacios de observación y acción
        self.observation_space = spaces.Dict({
            "volumen": spaces.Box(0.0, self.V_CLAIRE_MAX, shape=()),
            "hidrologia": spaces.Discrete(self.N_HIDRO),
            "tiempo": spaces.Discrete(52)
        })
        # turbinar en [0, V_MAX] continuo
        self.action_space = spaces.Box(0.0, self.V_CLAIRE_MAX, shape=())

    def reset(self, *, seed=None, options=None):
        self.v = self.V0
        self.t = 0
        self.h = self._inicial_hidrologia()
        info = {
            "volumen_inicial": self.v,
            "hidrologia_inicial": self.h,
            "tiempo_inicial": self.t
        }
        return self._get_obs(), info
    
    def step(self, action):
        # Validar que la acción esté en el espacio válido
        assert self.action_space.contains(action), f"Acción inválida: {action}. Debe estar en {self.action_space}"
        qt = float(action)
        
        # dinámica: v ← v − q − d + a
        self.v = self.v - qt - self._vertido(qt) + self._aportes()
        self.h = self._siguiente_hidrologia(self.t, self.h)
        self.t += 1

        # despacho: e_eolo + e_sol + e_bio + e_termico + e_hidro = dem + exp
        ingreso_exportacion, costo_termico = self._despachar(qt)

        # recompensa: −costo_termico + ingreso_exportacion
        reward = -costo_termico + ingreso_exportacion
        
        done = (self.t >= self.T_MAX)
        info = {
            "volumen": self.v,
            "tiempo": self.t,
            "hidrologia": self.h,
            "turbinado": qt,
            "vertido": self._vertido(qt),
            "ingreso_exportacion": ingreso_exportacion,
            "costo_termico": costo_termico,
        }
        return self._get_obs(), reward, done, False, info
    
    def render(self, mode='human'):
        if mode == 'human':
            print(f"Semana {self.t}/52:")
            print(f"  Volumen embalse: {self.v:.2f}/{self.V_CLAIRE_MAX}")
            print(f"  Estado hidrológico: {self.h}")
            print(f"  Porcentaje llenado: {(self.v/self.V_CLAIRE_MAX)*100:.1f}%")
            print("-" * 30)
        elif mode == 'rgb_array':
            # Retornar una imagen como array numpy para grabación
            pass
        elif mode == 'ansi':
            # Retornar string para mostrar en terminal
            return f"T:{self.t} V:{self.v:.1f} H:{self.h}"
        
    def _get_obs(self):
        # Mapeo de variables internas a observación del agente
        obs = {
            "volumen": self.v, 
            "hidrologia": self.h, 
            "tiempo": self.t
        }
        # Validar contra observation_space (opcional, útil para debug)
        assert self.observation_space.contains(obs), f"Observación inválida: {obs}. Debe estar en {self.observation_space}"
        return obs
    
# Auxiliares

# Leer archivo 
def leer_archivo(rutaArchivo, sep, header):
    return pd.read_csv(rutaArchivo, sep=sep, header=header, encoding='cp1252')   

In [11]:
leer_archivo(f"Datos\\MOP\\potencias_eolo1MW_PRUEBA_2025-7-8_14-45-28.xlt", "\t", 10)

,0,2023/1/1/0:0:0,1,0.44224705541228065,0.493083,0.412517,0.526647,0.253527,0.358369,0.541509,...,0.493083.7,0.673124.2,0.358369.4,0.354294.3,0.593072.3,0.571065.2,0.360604.3,0.309406196,0.271915.3,Unnamed: 118
0,0,2023/1/1/0:0:0,2,0.413243,0.574561,0.418614,0.437944,0.264032,0.474624,0.428149,...,0.574561,0.620167,0.474624,0.464717,0.539934,0.524838,0.346189,0.171236,0.244858,NaN
1,0,2023/1/1/0:0:0,3,0.402554,0.423246,0.353937,0.402926,0.164694,0.515344,0.423121,...,0.423246,0.616999,0.515344,0.583095,0.465066,0.504848,0.228754,0.143494,0.315355,NaN
2,0,2023/1/1/0:0:0,4,0.393911,0.427555,0.377427,0.416897,0.190947,0.521622,0.333754,...,0.427555,0.503216,0.521622,0.494288,0.503471,0.410001,0.290254,0.115406,0.461053,NaN
3,0,2023/1/1/0:0:0,5,0.370510,0.412310,0.429283,0.379306,0.348995,0.462042,0.394122,...,0.412310,0.302026,0.462042,0.453632,0.583365,0.419033,0.231322,0.048438,0.486545,NaN
4,0,2023/1/1/0:0:0,6,0.349229,0.318803,0.404337,0.350304,0.399228,0.364103,0.450903,...,0.318803,0.325763,0.364103,0.434719,0.646638,0.442911,0.258880,0.103827,0.447933,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17537,730,2024/12/31/0:0:0,19,0.416773,0.228645,0.375385,0.419401,0.140465,0.464566,0.419401,...,0.619364,0.228645,0.440403,0.108549,0.339973,0.424972,0.489569,0.319358,0.171797,NaN
17538,730,2024/12/31/0:0:0,20,0.484255,0.408989,0.440921,0.623742,0.186455,0.557112,0.623742,...,0.621220,0.408989,0.524815,0.080534,0.276700,0.505902,0.689424,0.351902,0.144737,NaN
17539,730,2024/12/31/0:0:0,21,0.533984,0.383619,0.607215,0.502508,0.225191,0.609254,0.502508,...,0.625610,0.383619,0.435338,0.156295,0.282408,0.608153,0.921528,0.398190,0.137428,NaN
17540,730,2024/12/31/0:0:0,22,0.570019,0.462133,0.600904,0.509004,0.155312,0.804627,0.509004,...,0.833722,0.462133,0.352335,0.228076,0.193775,0.734504,0.973321,0.487604,0.123704,NaN
